In [3]:
# Importing required libraries

import pandas as pd
from datetime import datetime
import re
import glob



# Defining the input file header

header=['PatientID', 'PatientMRN', 'LastName', 'FirstName', 'MiddleName', 'DateofBirth', 'Sex', 'Metric_Desc', 'Metric_Abbr', 'Result', 'UnitOfMeasure', 'CreateDTTM', 'PerformedDTTM', 'DecodedValue', 'QOCClassificationDE', 'AnswerDE', 'VitalValue', 'NormalizedValue', 'NumericFinding', 'Unit_Abbr', 'RecordedDTTM', 'QODE', 'Vital_Status', 'Mode_Name', 'ActivityHeaderID', 'ItemID', 'LoincLabCode', 'APM_Patient_ID', 'APM_MRN', 'APM_Patient_First_Name', 'APM_Patient_Last_Name', 'APM_Patient_MI', 'APM_Patient_Street1', 'APM_Patient_Street2', 'APM_Patient_City', 'APM_Patient_State', 'APM_Patient_Zip_Code', 'APM_Patient_Country', 'APM_Patient_Sex', 'APM_Patient_DOB', 'Patient_Home_Phone', 'Patient_Work_Phone', 'Patient_Work_Ext', 'Patient_Cell_Phone', 'Patient_Primary_Phone_Type', 'Patient_Primary_Phone_Number', 'Patient_Email']
# Read the Input File and convert to pandas dataframe
input_files = glob.glob('EPIC/Input.txt')
df_input = pd.read_csv(input_files[0], delimiter='|', header=None, names=header)
df_input.fillna('', inplace=True)
pd.set_option('display.max_columns', None)




# Read the Input File and convert to pandas dataframe

input_files = glob.glob('EPIC/Vitals_allscript.csv')
Vitals_allscript_df = pd.read_csv(input_files[0], delimiter=',',)
pd.set_option('display.max_columns', None)
# Vitals_allscript_df.head(5)
# df_input.columns.tolist()



# Apply strptime to each element of the 'CreateDTTM' column
df_input['reformated_CreateDTTM'] = pd.to_datetime(df_input['CreateDTTM'], format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%m-%d-%Y %H:%M')

# Convert 'reformated_CreateDTTM' back to datetime objects
df_input['reformated_CreateDTTM'] = pd.to_datetime(df_input['reformated_CreateDTTM'], format='%m-%d-%Y %H:%M')




df_input.head()
df_input['DateOfBirth'] = df_input['DateofBirth'].str.replace('-', '', regex=False)
df_input['CreateDTTM_converted'] = df_input['CreateDTTM'].str.replace('-', '', regex=False).str.replace(' ', '', regex=False).str.replace(':', '', regex=False).str.replace('.', '', regex=False).str[:-3]



lookup_table = {

    #MSH Segment
    "MSH-1": {"Source Column": None, "Default Value": None},
    "MSH-2": {"Source Column": None, "Default Value": None},
    "MSH-3": {"Source Column": None, "Default Value": "TouchWorks"},
    "MSH-4": {"Source Column": None, "Default Value": "MountainWest"},
    "MSH-5": {"Source Column": None, "Default Value": "Rhapsody^Rhapsody"},
    "MSH-6": {"Source Column": None, "Default Value": "Epic^Epic"},
    "MSH-7": {"Source Column": None, "Default Value": datetime.now().strftime("%Y%m%d%H%M%S")},
    "MSH-8": {"Source Column": None, "Default Value": None},
    "MSH-9": {"Source Column": None, "Default Value": "ORU^R01"},
    "MSH-10": {"Source Column": None, "Default Value": None},
    "MSH-11": {"Source Column": None, "Default Value": "P"},
    "MSH-12": {"Source Column": None, "Default Value": "2.5.1"},



    #PID segment

    "PID-1": {"Source Column": None, "Default Value": None},
    "PID-2": {"Source Column": None, "Default Value": None},
    "PID-3.1": {"Source Column": "PatientMRN", "Default Value": None},
    "PID-3.2": {"Source Column": None, "Default Value": None},
    "PID-3.3": {"Source Column": None, "Default Value": None},
    "PID-3.4": {"Source Column": None, "Default Value": None},
    "PID-3.5": {"Source Column": None, "Default Value": "TWSMAMRN"},
    "PID-4": {"Source Column": None, "Default Value": None},
    "PID-5.1": {"Source Column": "LastName", "Default Value": None},
    "PID-5.2": {"Source Column": "FirstName", "Default Value": None},
    "PID-5.3": {"Source Column": "MiddleName", "Default Value": None},
    "PID-6": {"Source Column": None, "Default Value": None},
    "PID-7": {"Source Column": "DateOfBirth", "Default Value": None},
    "PID-8": {"Source Column": "Sex", "Default Value": None},
    "PID-9": {"Source Column": None, "Default Value": None},
    "PID-10": {"Source Column": None, "Default Value": None},
    "PID-11": {"Source Column": None, "Default Value": None},
    "PID-12": {"Source Column": None, "Default Value": None},
    "PID-13": {"Source Column": None, "Default Value": None},
    "PID-14": {"Source Column": None, "Default Value": None},
    "PID-15": {"Source Column": None, "Default Value": None},
    "PID-16": {"Source Column": None, "Default Value": None},
    "PID-17": {"Source Column": None, "Default Value": None},
    "PID-18": {"Source Column": None, "Default Value": None},


    #OBR segment

    "OBR-1": {"Source Column": None, "Default Value": None},
    "OBR-2": {"Source Column": None, "Default Value": None},
    "OBR-3": {"Source Column": None, "Default Value": None},
    "OBR-4": {"Source Column": None, "Default Value": None},
    "OBR-5": {"Source Column": None, "Default Value": None},
    "OBR-6": {"Source Column": None, "Default Value": None},
    "OBR-7": {"Source Column": "RecordedDTTM", "Default Value": None},


    #OBX segment

    "OBX-1": {"Source Column": None, "Default Value": None},
    "OBX-2": {"Source Column": None, "Default Value": None},
    "OBX-3": {"Source Column": "Metric_Abbr", "Default Value": None},
    "OBX-4":{"Source Column":None,"Default Value":None},
    "OBX-5": {"Source Column": "Result", "Default Value": None},
    "OBX-6": {"Source Column": "Unit_Abbr", "Default Value": None},
    "OBX-7":{"Source Column":None,"Default Value":None},
    "OBX-8": {"Source Column": None, "Default Value": None},
    "OBX-9":{"Source Column":None,"Default Value":None},
    "OBX-10":{"Source Column":None,"Default Value":None},
    "OBX-11": {"Source Column":None, "Default Value": None},
    "OBX-12":{"Source Column":None,"Default Value":None},
    "OBX-13":{"Source Column":None,"Default Value":None},
    "OBX-14":{"Source Column":"?","Default Value":None}
}

def convert_to_int(value):

    try:
        return int(float(value))
    except (ValueError, TypeError):
        return value

pd.set_option('display.max_rows', None)

def get_value(field, row):

      source_column = lookup_table[field]["Source Column"]
      default_value = lookup_table[field]["Default Value"]

      if source_column:
          if isinstance(source_column, list):
              return "_".join([str(row.get(col, '')) for col in source_column])
          return row.get(source_column, default_value)
      return default_value

def parse_to_hl7(df_input, lookup_table):

    hl7_lines = []
    obx_counter = {}
    obr_counter = {}
    empty_list = []
    count = 0

    for index, row in df_input.iterrows():

      PatientMRN= row['PatientMRN']
      if PatientMRN not in obr_counter:
            obr_counter[PatientMRN] = 1
      else:
            obr_counter[PatientMRN] += 1

      #Create MSH segment

      msh_segment = [
          f"MSH|^~\&|{get_value('MSH-3', row)}|{get_value('MSH-4', row)}|{get_value('MSH-5', row)}|{get_value('MSH-6', row)}|{get_value('MSH-7', row)}||{get_value('MSH-9', row)}|{get_value('MSH-10', row)}|{get_value('MSH-11', row)}|{get_value('MSH-12', row)}"

              ]

      # Create PID segment
      pid_segment = [
                  f"PID|||{get_value('PID-3.1', row)}^^^^{get_value('PID-3.5', row)}||{get_value('PID-5.1', row)}^{get_value('PID-5.2', row)}||{get_value('PID-7', row)}|{get_value('PID-8', row)}"

                  ]
      # Create OBR segment
      obr_segment = [

                  f"OBR|{obr_counter[PatientMRN]}||||||{get_value('OBR-3', row)}|||||||||{convert_to_int(get_value('OBR-7', row))}"

              ]
        
      obx_segment = [
            f"OBX|{obx_counter[RecordedDTTM]}||{row_get('LoincLabCode', row)}^{row_get('Metric_Abbr', row)}^LN||{row_get('Result', row)}|{row_get('Unit_Abbr', row)}"
        ]

      hl7_message = "\n".join(msh_segment + pid_segment + obr_segment + obx_segment)

      hl7_lines.append(hl7_message)

    return hl7_lines
# Parse the input DataFrame to HL7 formatted output

hl7_output = parse_to_hl7(df_input, lookup_table)
# hl7_output


def sanitize_filename(filename):
    sanitized = re.sub(r'[<>:"/\\|?*]', '_', filename)
    sanitized = sanitized.replace(' ', '_')
    return sanitized


def remove_duplicate_segments(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    unique_segments = {'MSH': False, 'PID': False, 'ORC': False, 'OBR': False}
    with open(file_path, 'w') as file:
        for line in lines:
            segment_type = line.split('|')[0]
            if segment_type in unique_segments:
                if not unique_segments[segment_type]:
                    file.write(line)
                    unique_segments[segment_type] = True




file_paths = []
for index, row in df_input.iterrows():
    filename = f"{row['PatientMRN']}"
    sanitized_filename = sanitize_filename(filename)
    file_path = f"EPIC/{sanitized_filename}.hl7"
    file_paths.append(file_path)
    with open(file_path, 'w') as file:
        for line in hl7_output:
                file.write(line + "\n")

for file_path in set(file_paths):
    remove_duplicate_segments(file_path)



print(hl7_output)

<>:160: SyntaxWarning: invalid escape sequence '\&'
<>:160: SyntaxWarning: invalid escape sequence '\&'
/var/folders/kg/dh63qncd4wg_4xfns1k_65jc0000gp/T/ipykernel_87155/3858842577.py:160: SyntaxWarning: invalid escape sequence '\&'
  f"MSH|^~\&|{get_value('MSH-3', row)}|{get_value('MSH-4', row)}|{get_value('MSH-5', row)}|{get_value('MSH-6', row)}|{get_value('MSH-7', row)}||{get_value('MSH-9', row)}|{get_value('MSH-10', row)}|{get_value('MSH-11', row)}|{get_value('MSH-12', row)}"
/var/folders/kg/dh63qncd4wg_4xfns1k_65jc0000gp/T/ipykernel_87155/3858842577.py:160: SyntaxWarning: invalid escape sequence '\&'
  f"MSH|^~\&|{get_value('MSH-3', row)}|{get_value('MSH-4', row)}|{get_value('MSH-5', row)}|{get_value('MSH-6', row)}|{get_value('MSH-7', row)}||{get_value('MSH-9', row)}|{get_value('MSH-10', row)}|{get_value('MSH-11', row)}|{get_value('MSH-12', row)}"


IndexError: list index out of range

In [ ]:
import csv
from datetime import datetime

# Define the correct header manually based on the structure of your file
headers = ['PatientId', 'Patient MRN', 'LastName', 'FirstName', 'MiddleName', 'DateOfBirth', 'Sex', 'Metric_Desc', 
           'Metric_Abbr', 'Result', 'UnitOfMeasure', 'CreateDTTM', 'PerformedDTTM', 'DecodedValue', 
           'QOClassificationDE', 'AnswerDE', 'VitalValue', 'NormalizedValue', 'NumericFinding', 'Unit_Abbr', 
           'RecordedDTTM', 'QODE', 'Vital_Status', 'Mode_Name', 'ActivityHeaderID', 'ItemID', 'LoincLabCode', 
           'APM_Patient_ID', 'APM_MRN', 'APM_Patient_First_Name', 'APM_Patient_Last_Name', 'APM_Patient_MI', 
           'APM_Patient_Street1', 'APM_Patient_Street2', 'APM_Patient_City', 'APM_Patient_State', 'APM_Patient_Zip_Code', 
           'APM_Patient_Country', 'APM_Patient_Sex', 'APM_Patient_DOB', 'Patient_Home_Phone', 'Patient_Work_Phone', 
           'Patient_Work_Ext', 'Patient_Cell_Phone', 'Patient_Primary_Phone_Type', 'Patient_Primary_Phone_Number', 
           'Patient_Emailx']

def create_hl7_message(row):
    # Create MSH segment
    msh_segment = f'MSH|^~\\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|{datetime.now().strftime("%Y%m%d%H%M%S")}||ORU^R01|{row["Patient MRN"]}_{row["ActivityHeaderID"]}_{row["PerformedDTTM"].replace("-", "").replace(":", "").replace(" ", "")}|P|2.5.1'

    # Create PID segment
    pid_segment = f'PID|||{row["Patient MRN"]}^^^^TWSMAMRN||{row["LastName"]}^{row["FirstName"]}^{row["MiddleName"]}||{row["DateOfBirth"].replace("-", "")}|{row["Sex"]}|||{row["APM_Patient_Street1"]}^^{row["APM_Patient_City"]}^{row["APM_Patient_State"]}^{row["APM_Patient_Zip_Code"]}'

    # Create OBR segment
    obr_segment = f'OBR|1||||||{row["PerformedDTTM"].replace("-", "").replace(" ", "").replace(":", "")}'

    # Create OBX segment (for observations)
    obx_segment = f'OBX|1||{row["LoincLabCode"]}^{row["Metric_Abbr"]}^LN||{row["Result"]}|{row["UnitOfMeasure"]}'

    # Combine all segments
    hl7_message = f'{msh_segment}\n{pid_segment}\n{obr_segment}\n{obx_segment}\n'
    
    return hl7_message

def convert_to_hl7(input_file, output_file):
    # Open the CSV file and the output file
    with open(input_file, mode='r') as infile, open(output_file, mode='w') as outfile:
        csv_reader = csv.DictReader(infile, delimiter='|', fieldnames=headers)
        
        # Skip the first row since it’s the actual data row, not a header
        next(csv_reader)
        
        # Process each row to generate HL7 messages
        for row in csv_reader:
            hl7_message = create_hl7_message(row)
            outfile.write(hl7_message + '\n')

# Example usage inside a Jupyter notebook
input_file = 'Input.txt'  # Replace with your input file path
output_file = 'output.hl7'  # Replace with your desired output file path
convert_to_hl7(input_file, output_file)

# Display confirmation that the process is done
print(f"HL7 messages have been written to {output_file}.")
